In [1]:
import pandas as pd 
import pymongo
from dataProcessing import agroEcoData, contextData, legumeData, pairwiseData, participatoryMatrixData

## Connecting to DB

In [2]:
client = pymongo.MongoClient("localhost", 27017)
db = client["legume-choice"]
print(db.name)
# Extracting the projects data
projectsData =  db["projects"]
# Finding the number of projects
print(projectsData.count_documents({}))

legume-choice
3


## Extracting Data

In [3]:
# Extacting all projects
allProjects = []
for project in projectsData.find():
        allProjects.append(project)

# Example querying from the dataset
#allProjects[0]["rawdata"]["agroEcoData"]


## Extracting Agro Ecological Scores for all Projects

In [6]:
# Tabular AgroEco scores

# def ExtractProjectAgroEcoData(project):
#     projectID =  project["projectID"]
#     projectName =  project["rawdata"]["projectInfo"]["projectName"]
#     country =  project["rawdata"]["projectInfo"]["country"]
#     agroEcoData = project["rawdata"]["agroEcoData"]["biofilters"]

#     row={"country":country,"projectName":projectName,"projectID":projectID,}


#     for index in range(0,len(agroEcoData)):
#         row[agroEcoData[index]["label"]]=agroEcoData[index]["value"]
#     return row
# def ExtractAllAgroEcoData(projects):
#     agroEcoData=[]
#     for project in projects:
#         agroEcoData.append(ExtractProjectAgroEcoData(project=project))
#     return pd.DataFrame(agroEcoData)



agroEcoData.ExtractProjectAgroEcoData(allProjects[0])
agroEcoData.ExtractAllAgroEcoData(allProjects)


{'country': '',
 'projectName': 'wqeaafd',
 'projectID': 'd4e74b50-6b07-4bb2-9be7-af7f52e4947d',
 'rainfall': 2175,
 'temp': 14.5,
 'alt': 1845,
 'soilpH': 8.5}

In [ ]:
agroEcoSingleProj = pd.DataFrame(columns=list(agroEcoData.ExtractProjectAgroEcoData(allProjects[0]).keys()))
agroEcoSingleProj.loc[0]=agroEcoData.ExtractProjectAgroEcoData(allProjects[0]).values()
agroEcoSingleProj

## Extracting all Data for Context Scores

In [ ]:
def subsetScoresByVariables(scores, attributeLabel, participantLabel, typologyLabel, average):

    if (average==False):
        for score in scores:
            if (score["scoreType"]=="individual" and 
            score["attribute"]["label"]==attributeLabel and 
            score["participant"]["label"]==participantLabel and 
            score["typology"]["label"]==typologyLabel):
                 return score["score"]

    if (average==True):
        for score in scores:
            if (score["scoreType"]=="average" and 
            score["attribute"]["label"]==attributeLabel): 
                return score["score"]

    raise ValueError("Did not identify correct subset")
    

def ScoresForAllAttributes(scores, attributes, participant, typology, average):
    listOfScores=[]
    for attribute in attributes:
        listOfScores.append(subsetScoresByVariables(scores=scores, 
        attributeLabel=attribute["label"], 
        participantLabel=participant["label"], 
        typologyLabel=typology["label"], 
        average=average))

    return listOfScores


def projectContextScores(project):

    # Initialising Variables
    projectID =  project["projectID"]
    projectName =  project["rawdata"]["projectInfo"]["projectName"]
    country =  project["rawdata"]["projectInfo"]["country"]


    allScores = project["rawdata"]["contextScores"]["scores"]
    attributes = project["rawdata"]["contextScores"]["attributes"]
    participants = project["rawdata"]["contextScores"]["participants"]
    typologies = project["rawdata"]["contextScores"]["typologies"]

    # Creating column names for dataframe
    typologyColumns = [ "typ"+typology["label"] for typology in typologies]
    participantColumns = [participant["label"] for participant in participants]
    allColumns = [typ+particip for typ in typologyColumns for particip in participantColumns]

    # Initialising empty array of scores
    scoresToReturn={}
    column=0
    scoresToReturn["country"] = [country for attribute in attributes]
    scoresToReturn["projectName"] = [projectName for attribute in attributes]
    scoresToReturn["projectID"] = [projectID for attribute in attributes]
    for typology in typologies:
        for participant in participants:
            scoresToReturn[allColumns[column]]=ScoresForAllAttributes(scores=allScores, 
                                                                        attributes=attributes,
                                                                        participant=participant, 
                                                                        typology=typology, 
                                                                        average=False )
            column+=1
    
    scoresToReturn["average"]=ScoresForAllAttributes(scores=allScores, 
                                                        attributes=attributes,
                                                        participant=participants[1], 
                                                        typology=participants[1], 
                                                        average=True )
    return scoresToReturn

def ContextScoresAllProjectsList(projects):
    projectsCombined=[]
    #projectIndex=0
    for project in projects:
        projectsCombined.append(projectContextScores(project=project))
        #projectIndex+=1
    return(projectsCombined)

def ContextScoresAllProjects(projects):
    contextScoresList = ContextScoresAllProjectsList(allProjects)
    if len(contextScoresList)==1:
        return pd.DataFrame(contextScoresList)
    if len(contextScoresList)>1:
        contextScoresDF=pd.DataFrame(contextScoresList[0])
        for scoreItem in contextScoresList[1:]:
            contextScoresDF = contextScoresDF.append(pd.DataFrame(scoreItem))
    return contextScoresDF
ContextScoresAllProjects(projects=allProjects)


## Extracting Data for PairWise Rankings

### Individual Selections

In [ ]:
def pairWiseSelectionsProjectGender(project,gender):

    selections = project["rawdata"]["pairWiseScores"][gender]["pairwiseSelection"]
    projectName= project["rawdata"]["projectInfo"]["projectName"]
    projectID= project["projectID"]
    country =  project["rawdata"]["projectInfo"]["country"]


    selectionsToReturn=[]

    for selection in selections:
        selectionToAppend={
            "country": country,
            "projectName": projectName,
            "projectID":projectID,
            "option1": selection["funct1"]["name"],
            "option2": selection["funct2"]["name"],
            gender+"Choice": selection["value"]["name"],
        }
        selectionsToReturn.append(selectionToAppend)
    return pd.DataFrame(selectionsToReturn)

def pairWiseSelectionsProject(project):
    male=pairWiseSelectionsProjectGender(project=project,gender="male")
    female=pairWiseSelectionsProjectGender(project=project,gender="female")

    allSelections = male
    allSelections["femaleChoice"] = female["femaleChoice"]
    return allSelections

def pairWiseSelectionsAllProjects(projects):
    if len(projects)==1:
        return pairWiseSelectionsProject(projects)
    if len(projects)>1:
        pairWiseScoresDF=pairWiseSelectionsProject(projects[0])
        for project in projects[1:]:
            pairWiseScoresDF = pairWiseScoresDF.append(pairWiseSelectionsProject(project))
    return pairWiseScoresDF
pairWiseSelectionsAllProjects(projects=allProjects)



### Averages

In [ ]:
def pairWiseSummaryScores(project):
    country =  project["rawdata"]["projectInfo"]["country"]
    projectID = project["projectID"]
    projectName = project["rawdata"]["projectInfo"]["projectName"]
    maleTotals = project["rawdata"]["pairWiseScores"]["male"]["totals"]
    femaleTotals = project["rawdata"]["pairWiseScores"]["female"]["totals"]
    averages = project["rawdata"]["pairWiseScores"]["averages"]


    pairWiseAverages = []
    for scoreIndex in range(0,len(maleTotals)):
        scoresToAppend = {
            "country": country,
            "projectName":projectName,
            "projectID":projectID,
            "attribute": maleTotals[scoreIndex]["attribute"]["name"],
            "countMale": maleTotals[scoreIndex]["value"],
            "countFemale": femaleTotals[scoreIndex]["value"],
            "average": averages[scoreIndex]["value"],
        }
        pairWiseAverages.append(scoresToAppend)

    return pd.DataFrame(pairWiseAverages)

def pairWiseSelectionSummaryAllProjects(projects):
    if len(projects)==1:
        return pairWiseSummaryScores(projects)
    if len(projects)>1:
        pairWiseSummaryScoresDF=pairWiseSummaryScores(projects[0])
        for project in projects[1:]:
            pairWiseSummaryScoresDF = pairWiseSummaryScoresDF.append(pairWiseSummaryScores(project))
    return pairWiseSummaryScoresDF
pairWiseSelectionSummaryAllProjects(projects=allProjects)


## Participatory Matrix Scores

In [ ]:

# Extracting farmer data
def participatoryMatrixScoresProject(project):
    #project=allProjects[0]
    country =  project["rawdata"]["projectInfo"]["country"]
    projectID = project["projectID"]
    projectName = project["rawdata"]["projectInfo"]["projectName"]
    farmers = project["rawdata"]["participatoryMatrixScores"]["farmers"]
    farmersClean=[]
    for farmer in farmers:
        newFarmer={}
        newFarmer["country"]=country
        newFarmer["projectID"]=projectID
        newFarmer["projectName"]=projectName
        newFarmer["gender"]=farmer["gender"]
        newFarmer["typology"]=farmer["typology"]
        
        for selection in farmer["selections"]:
            newFarmer[selection["label"]]=selection["score"]
        newFarmer["total"]=farmer["total"]
        farmersClean.append(newFarmer)
    return pd.DataFrame(farmersClean)


def participatoryMatrixAllProjects(projects):
    if len(projects)==1:
        return participatoryMatrixScoresProject(projects)
    if len(projects)>1:
        participatoryMatrixScoresDF=participatoryMatrixScoresProject(projects[0])
        for project in projects[1:]:
            participatoryMatrixScoresDF = participatoryMatrixScoresDF.append(participatoryMatrixScoresProject(project))
    return participatoryMatrixScoresDF
participatoryMatrixAllProjects(projects=allProjects)

In [ ]:
# projectIndex = 0
# project = allProjects[projectIndex]
# legumes = project["rawdata"]["results"]["legumes"]


def extractLegumeResults(legume, country, projectName, projectID):
    contextVariables = ["land",
            "labour",
            "seed",
            "inp-serv",
            "knowl-skill",
            "water",
            "markets"]

    functionVariables=["food",
            "feed",
            "income",
            "erosion-control",
            "fuel",
            "soil-fertility"]

    legumeVariables = ["name",
            "type",
            "food",
            "feed",
            "income",
            "erosion-control",
            "fuel",
            "soil-fertility",
            "land",
            "labour",
            "seed",
            "inp-serv",
            "knowl-skill",
            "water",
            "markets",
            "agroecology-source"]

    maxMinVariables=["rainfall",
            "temp",
            "alt",
            "soilpH"]

    legumeData = {
        "country":country, 
        "projectName": projectName, 
        "projectID":projectID,
        "legumeName":legume["name"]

    }
    #legume = legumes[0]
    for variable in  legumeVariables:
        #print(variable)
        if variable in contextVariables:
            legumeData["context_"+variable]=legume[variable]
        if variable in functionVariables:
            legumeData["function_"+variable]=legume[variable]
    for variable in maxMinVariables:
        legumeData["agroEco_"+variable+"_min"]=legume[variable][0]["min"]
        legumeData["agroEco_"+variable+"_max"]=legume[variable][0]["max"]

    for agroEcoScore in legume["results"]["agroEcoFit"]:
        if "agroEcoFilter" in agroEcoScore.keys():
            scoreAttribute=agroEcoScore["agroEcoFilter"]["label"]
            legumeData["agroEcoScore_"+scoreAttribute]=agroEcoScore["score"]
        if "overallFit" in agroEcoScore.keys():
            legumeData["OVERALL_agroEcoScore"]=agroEcoScore["overallFit"]
        if "overallRank" in agroEcoScore.keys():
            legumeData["OVERALL_agroEcoRank"]=agroEcoScore["overallRank"]

    for contextScore in legume["results"]["contextFit"]:
        if "attribute" in contextScore.keys():
            scoreAttribute=contextScore["attribute"]["label"]
            legumeData["contextScore_"+scoreAttribute]=contextScore["score"]
        if "overallFit" in contextScore.keys():
            legumeData["OVERALL_contextScore"]=contextScore["overallFit"]
        if "overallRank" in contextScore.keys():
            legumeData["OVERALL_contextRank"]=contextScore["overallRank"]

    for functionScore in legume["results"]["functionFit"]:
        if "legumeFunction" in functionScore.keys():
            scoreAttribute=functionScore["legumeFunction"]["label"]
            legumeData["functionScore"+scoreAttribute]=functionScore["score"]
        if "overallFit" in functionScore.keys():
            legumeData["OVERALL_functionScore"]=functionScore["overallFit"]
        if "overallRank" in functionScore.keys():
            legumeData["OVERALL_functionRank"]=functionScore["overallRank"]
    
    return legumeData





In [ ]:
#extractLegumeResults(legumes[0])

def extractAllLegumeData(legumes, country, projectName, projectID):
    legumeData = []

    for legume in legumes:
        legumeData.append(extractLegumeResults(legume=legume, 
                                                country=country, 
                                                projectName=projectName, 
                                                projectID=projectID))

    return pd.DataFrame(legumeData)


In [ ]:
    
country =  project["rawdata"]["projectInfo"]["country"]
projectID = project["projectID"]
projectName = project["rawdata"]["projectInfo"]["projectName"]
legumes = project["rawdata"]["results"]["legumes"]
extractAllLegumeData(legumes=legumes, country=country, projectName=projectName, projectID=projectID)




In [ ]:
agroEcoData.ExtractProjectAgroEcoData??